# Smartnoise Synth

In [58]:
from lomas_client import Client

In [59]:
APP_URL = "http://lomas_server"
USER_NAME = "Dr. Antartica"
DATASET_NAME = "PENGUIN"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

In [60]:
penguin_metadata = client.get_dataset_metadata()
penguin_metadata

{'max_ids': 1,
 'row_privacy': True,
 'censor_dims': False,
 'columns': {'species': {'type': 'string',
   'cardinality': 3,
   'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
  'island': {'type': 'string',
   'cardinality': 3,
   'categories': ['Torgersen', 'Biscoe', 'Dream']},
  'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
  'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
  'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
  'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
  'sex': {'type': 'string',
   'cardinality': 2,
   'categories': ['MALE', 'FEMALE']}}}

### DPCTGAN

In [61]:
cost = client.estimate_smartnoise_synth_cost(
    synth_name="dpctgan",
    epsilon= 1.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {
        "batch_size": 2,
        "epochs": 5
    },
    nullable = True,
    table_transformer_style = "gan",
    constraints = "",
)
cost

{'epsilon_cost': 0.11916262670256472, 'delta_cost': 0.00015673368198174188}

In [84]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon=0.1,
    delta=1e-5,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 2,
        "epochs": 2
    },
    nullable = True,
    table_transformer_style = "gan",
    dummy = False,
    constraints = "",
    return_model = False,
    nb_samples = 10
)

In [85]:
df = res["query_response"]
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Chinstrap,Biscoe,64.997956,21.623620,249.360874,2000.354499,None
1,Gentoo,Biscoe,64.981632,21.546940,249.185118,2007.097274,None
2,Adelie,Biscoe,64.996121,20.388359,249.373353,2001.765341,FEMALE
3,Gentoo,Biscoe,64.999406,20.156239,249.783587,2002.305210,None
4,Chinstrap,Biscoe,64.987848,21.345602,249.655151,2003.287792,MALE


In [ ]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon=0.1,
    delta=1e-5,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 2,
        "epochs": 2
    },
    nullable = True,
    table_transformer_style = "gan",
    dummy = False,
    constraints = "",
    return_model = True,
    nb_samples = 100
)
res

In [ ]:
model = res["query_response"]
model.sample(5)

In [41]:
model.sample(5).columns

Index(['age', 'sex', 'educ', 'race', 'income', 'married'], dtype='object')

# Smartnoise Synth

In [9]:
import pandas as pd
from snsynth import Synthesizer
from snsynth.transform.table import TableTransformer
from snsynth.transform import AnonymizationTransformer, BinTransformer, ChainTransformer, LabelTransformer, MinMaxTransformer, OneHotEncoder

In [10]:
def get_constraints(style, nullable):
    if style == "gan":
        constraints = {
           'species': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
           'island': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
           'bill_length_mm': MinMaxTransformer(lower=30.0, upper=65.0, nullable=True),
           'bill_depth_mm': MinMaxTransformer(lower=13.0, upper=23.0, nullable=True),
           'flipper_length_mm': MinMaxTransformer(lower=150.0, upper=250.0, nullable=True),
           'body_mass_g': MinMaxTransformer(lower=2000.0, upper=7000.0, nullable=True),
           'sex': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
        }
    else:
        constraints = {
           'species': LabelTransformer(nullable=nullable),
           'island': LabelTransformer(nullable=nullable),
           'bill_length_mm': BinTransformer(lower=150.0, upper=250.0, bins=20, nullable=True),
           'bill_depth_mm': BinTransformer(lower=13.0, upper=23.0, bins=20, nullable=True),
           'flipper_length_mm': BinTransformer(lower=150.0, upper=250.0, bins=20, nullable=True),
           'body_mass_g': BinTransformer(lower=2000.0, upper=7000.0, bins=20, nullable=True),
           'sex': LabelTransformer(nullable=nullable),
        }
    return constraints

In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")
df.shape[0]

344

In [12]:
style = "gan"
nullable = True

In [13]:
constraints = get_constraints(style, nullable)
transformer = TableTransformer.create(data=df, style=style, nullable=nullable, constraints=constraints)
transformer

In [14]:
dpctgan_params = {"batch_size": 2, "epochs": 5}
params = dpctgan_params

In [15]:
model = Synthesizer.create(
    synth="dpctgan", 
    epsilon=1.0, 
    verbose=True,
    **params
)
model.fit(data=df, transformer=transformer, preprocessor_eps=0.0, nullable=True)

/usr/local/lib/python3.11/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Epoch 1, Loss G: 0.6737, Loss D: 1.4400
epsilon is 0.0584807515514389, alpha is 63.0
Epoch 2, Loss G: 0.6731, Loss D: 1.4554
epsilon is 0.07365122033922036, alpha is 63.0
Epoch 3, Loss G: 0.7123, Loss D: 1.4747
epsilon is 0.08882168912700181, alpha is 63.0
Epoch 4, Loss G: 0.6891, Loss D: 1.4357
epsilon is 0.10399215791478325, alpha is 63.0
Epoch 5, Loss G: 0.6478, Loss D: 1.4073
epsilon is 0.11916262670256472, alpha is 63.0


In [17]:
model.delta

0.00015673368198174188

In [19]:
model.epsilon_list[-1]

0.11916262670256472

In [118]:
vars(model)

{'_embedding_dim': 128,
 '_generator_dim': (256, 256),
 '_discriminator_dim': (256, 256),
 '_generator_lr': 0.0002,
 '_generator_decay': 1e-06,
 '_discriminator_lr': 0.0002,
 '_discriminator_decay': 1e-06,
 '_batch_size': 2,
 '_discriminator_steps': 1,
 '_verbose': True,
 '_epochs': 5,
 'pac': 1,
 'sigma': 5,
 'disabled_dp': False,
 'delta': 0.00015673368198174188,
 'max_per_sample_grad_norm': 1.0,
 'epsilon': 1.0,
 'epsilon_list': [0.0584807515514389,
  0.07365122033922036,
  0.08882168912700181,
  0.10399215791478325,
  0.11916262670256472],
 'alpha_list': [63.0, 63.0, 63.0, 63.0, 63.0],
 'loss_d_list': [tensor(1.4195, grad_fn=<AddBackward0>),
  tensor(1.4409, grad_fn=<AddBackward0>),
  tensor(1.4579, grad_fn=<AddBackward0>),
  tensor(1.4930, grad_fn=<AddBackward0>),
  tensor(1.4769, grad_fn=<AddBackward0>)],
 'loss_g_list': [tensor(0.7057, grad_fn=<AddBackward0>),
  tensor(0.6949, grad_fn=<AddBackward0>),
  tensor(0.6525, grad_fn=<AddBackward0>),
  tensor(0.5930, grad_fn=<AddBackwar

In [111]:
m1 = [0.0584807515514389,
  0.07365122033922036,
  0.08882168912700181,
  0.10399215791478325,
  0.11916262670256472]
m2 = [0.0584807515514389,
  0.07365122033922036,
  0.08882168912700181,
  0.10399215791478325,
  0.11916262670256472]
m3 = [0.0584807515514389,
  0.07365122033922036,
  0.08882168912700181,
  0.10399215791478325,
  0.11916262670256472]
delta_1 = 0.00015673368198174188
delta_2 = 0.00015673368198174188,

In [113]:
m1 == m3

True

In [16]:
res = client.smartnoise_synth_query(
    synth_name="mwem",
    epsilon= 2.0,
    #delta = 0.0001,
    select_cols = [],
    synth_params = {},
    nullable = True,
    table_transformer_style = "gan",
    dummy = True,
    constraints = "",
)
res

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
model = res["query_response"]

In [ ]:
model.sample(5)

In [ ]:
model.__class__.__name__

In [53]:
# Smartnoise-Synth
example_smartnoise_synth_cost = {
    "dataset_name": "apjnflia",
    "synth_name": "g",
    "epsilon": 0.1,
    "delta": 0.001,
    "select_cols": [],
    "synth_params": {
        "embedding_dim": 128,
        "generator_dim": (256, 256),
        "discriminator_dim": (256, 256),
        "batch_size": 50,
        "epochs": 5,
    },
    "nullable": True,
    "table_transformer_style": "gan",
    "constraints": "",
}
example_smartnoise_synth_query = dict(example_smartnoise_synth_cost)
example_smartnoise_synth_query["return_model"] = True
example_smartnoise_synth_query["condition"] = ""
example_smartnoise_synth_query["nb_samples"] = 200

In [54]:
from pydantic import BaseModel, Field
from typing import List, Optional

class SmartnoiseSynthCostModel(BaseModel):
    """Model input for a smarnoise-synth cost"""

    dataset_name: str
    synth_name: str
    epsilon: float = Field(..., gt=0, le=10)
    delta: Optional[float] = None
    select_cols: List
    synth_params: dict
    nullable: bool
    table_transformer_style: str
    constraints: str


class SmartnoiseSynthQueryModel(SmartnoiseSynthCostModel):
    """Dummy Model input for a smarnoise-synth query"""

    return_model: bool
    condition: str
    nb_samples: int


In [55]:
query_json = SmartnoiseSynthQueryModel.model_validate(example_smartnoise_synth_query)

In [56]:
if query_json.condition:
    print("ok")

In [57]:
from snsynth.transform import ChainTransformer, LabelTransformer, MinMaxTransformer, OneHotEncoder
from smartnoise_synth_logger import serialise_constraints

constraints = {
   'species': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'island': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'bill_length_mm': MinMaxTransformer(lower=30.0, upper=65.0, nullable=True),
   'bill_depth_mm': MinMaxTransformer(lower=13.0, upper=23.0, nullable=True),
   'flipper_length_mm': MinMaxTransformer(lower=150.0, upper=250.0, nullable=True),
   'body_mass_g': MinMaxTransformer(lower=2000.0, upper=7000.0, nullable=True),
   'sex': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
}

In [51]:
c = serialise_constraints(constraints)
c

'{"module": "smartnoise-synth", "version": "1.0.4", "constraints": {"species": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "island": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "bill_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 30.0, "upper": 65.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "bill_depth_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 13.0, "upper": 23.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "flipper_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 150.0, "upper": 250.0, "epsilon": 0.0, "negative": tr

In [55]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 2
    },
    nullable = True,
    constraints = constraints,
    table_transformer_style = "gan",
    dummy = True
)
res

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f2e0be779d0>}

In [57]:
model = res["query_response"]
model.sample(2)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,64.979197,13.000297,150.001132,6999.272376,FEMALE
1,Adelie,Torgersen,64.995704,13.000055,150.000179,6999.830872,FEMALE


## Error 1: test_smartnoise_synth_query_transformer_type

In [42]:
res = client.smartnoise_synth_query(
    synth_name="dpctgan",
    epsilon= 2.0,
    delta = 0.0001,
    select_cols = [],
    synth_params = {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 2
    },
    nullable = True,
    table_transformer_style = "cube",
    dummy = True
)
res

/usr/local/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


{'query_response': <snsynth.pytorch.nn.dpctgan.DPCTGAN at 0x7f2e0c4a7450>}

In [43]:
model_full = res["query_response"]

In [44]:
df = model_full.sample(10)

TypeError: list indices must be integers or slices, not numpy.float32

In [32]:
vars(model_full).keys()

dict_keys(['_embedding_dim', '_generator_dim', '_discriminator_dim', '_generator_lr', '_generator_decay', '_discriminator_lr', '_discriminator_decay', '_batch_size', '_discriminator_steps', '_verbose', '_epochs', 'pac', 'sigma', 'disabled_dp', 'delta', 'max_per_sample_grad_norm', 'epsilon', 'epsilon_list', 'alpha_list', 'loss_d_list', 'loss_g_list', 'verbose', 'loss', '_device', '_transformer', '_data_sampler', '_generator'])

In [48]:
#vars(model_full)

In [37]:
model_full.delta

0.0001

In [27]:
df = model_full.sample(5)

In [34]:
list(df.columns) ==['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex']

True

In [16]:
from snsynth.transform.type_map import TypeMap

In [ ]:
TypeMap.get_transformers

In [12]:
import pandas as pd
from snsynth import Synthesizer
from snsynth.transform import (
    AnonymizationTransformer,
    BinTransformer,
    ChainTransformer,
    #DateTimeTransformer,
    LabelTransformer,
    MinMaxTransformer,
    OneHotEncoder,
)
from snsynth.transform.table import TableTransformer

from typing import Dict, List, Optional

In [15]:
from snsynth.transform.datetime import DateTimeTransformer

In [ ]:
Synthesizer.list_synthesizers()

In [ ]:
from pydantic import BaseModel, Field
class SmartnoiseSynthModel(BaseModel):
    """Model input for a smarnoise-synth query"""

    dataset_name: str
    synth_name: str
    epsilon: float = 0.0
    delta: float = 0.0
    select_cols: List = []
    synth_params: dict = {}
    mul_matrix: List = []
    nullable: bool = True
    condition: Optional[str] = None
    nb_samples: Optional[int] = None
    table_transformer_style: str = "gan"

In [ ]:
from enum import StrEnum
class SSynthSynthesizer(StrEnum):
    """Synthesizer models for smartnoise synth"""

    # Marginal Synthesizer
    AIM = "aim"
    MWEM = "mwem"
    MST = "mst"
    PAC_SYNTH = "pacsynth"

    # Neural Network Synthesizer
    DP_CTGAN = "dpctgan"
    PATE_CTGAN = "patectgan"
    PATE_GAN = "pategan"  # no documentation
    DP_GAN = "dpgan"  # no documentation

    # Hybrid Synthesizer
    QUAIL = "quail"


class SSynthTableTransStyle(StrEnum):
    """Transformer style for smartnoise synth"""

    GAN = "gan"
    CUBE = "cube"


class SSynthColumnType(StrEnum):
    """Type of columns for SmartnoiseSynth transformer pre-processing"""

    PRIVATE_ID = "private_id"
    CATEGORICAL = "categorical"
    CONTINUOUS = "continuous"
    ORDINAL = "ordinal"
    DATETIME = "datetime"


SSYNTH_PRIVATE_COLUMN = "uuid"
SSYNTH_DEFAULT_NB_SAMPLES = 100

In [ ]:
def _preprocess_data(
        private_data: pd.DataFrame,
        query_json: dict,
    ) -> pd.DataFrame:
        """
        Preprocess the data based on the query parameters.

        Args:
            private_data (pd.DataFrame): Private data to be preprocessed
            query_json (dict): (SmartnoiseSynthModelCost): JSON request object
                select_cols (List[str]): List of columns to select
                mul_matrix (List): Multiplication matrix for columns aggregations

        Returns:
            pd.DataFrame: Preprocessed private data
        """
        if query_json.select_cols:
            try:
                private_data = private_data[query_json.select_cols]
            except KeyError as e:
                raise ValueError(
                    "Error while selecting provided select_cols: " + str(e)
                ) from e

        if query_json.mul_matrix:
            try:
                np_matrix = np.array(query_json.mul_matrix)
                mul_private_data = private_data.to_numpy().dot(np_matrix.T)
                private_data = pd.DataFrame(mul_private_data)
            except ValueError as e:
                raise ValueError(
                    f"Failed to multiply provided mul_matrix: {(str(e))}"
                ) from e
        return private_data

In [ ]:
def _categorize_column(data: dict) -> str:
    """
    Categorize the column based on its metadata.

    Args:
        data (dict): Metadata of the column.

    Returns:
        str: Category of the column.
    """
    match data["type"]:
        case "string" | "boolean":
            return SSynthColumnType.CATEGORICAL
        case "int" | "float":
            if "lower" in data.keys():
                return SSynthColumnType.CONTINUOUS
            if "cardinality" in data.keys():
                return SSynthColumnType.CATEGORICAL
            return SSynthColumnType.ORDINAL
        case "datetime":
            return SSynthColumnType.DATETIME
        case _:
            raise ValueError(
                f"Unknown column type in metadata: {data['type']}"
            )

def _get_column_by_types(
    metadata,
    select_cols: List[str],
) -> Dict[str, List[str]]:
    """
    Sort the column in categories based on their types and metadata

    Args:
        metadata (Metadata): Metadata of the dataset
        select_cols (List[str]): List of columns to select

    Returns:
        Dict[str, List[str]]: Dictionnary of list of columns by categories
    """
    col_categories: Dict[str, List[str]] = {
        SSynthColumnType.CATEGORICAL: [],
        SSynthColumnType.CONTINUOUS: [],
        SSynthColumnType.DATETIME: [],
        SSynthColumnType.ORDINAL: [],
        SSynthColumnType.PRIVATE_ID: [],
    }
    for col_name, data in metadata["columns"].items():
        if select_cols and col_name not in select_cols:
            continue
        
        if "private_id" in data.keys():
            col_categories[SSynthColumnType.PRIVATE_ID].append(col_name)
            continue

        # Sort the column in categories based on their types and metadata
        category = _categorize_column(data)
        col_categories[category].append(col_name)

    return col_categories

def _prepare_data_transformer(
    metadata,
    private_data: pd.DataFrame,
    query_json: dict,
):
    """
    Creates the transformer based on the metadata
    The transformer is used to transform the data before synthesis and then
    reverse the transformation after synthesis.
    See https://docs.smartnoise.org/synth/transforms/index.html for documentation
    See https://github.com/opendp/smartnoise-sdk/blob/main/synth/snsynth/
        transform/type_map.py#L40 for get_transformer() method taken as basis.

    Args:
        metadata (Metadata): Metadata of the dataset
        private_data
        query_json (SmartnoiseSynthModelCost): JSON request object for the query
            select_cols (List[str]): List of columns to select
            nullable (bool): True is the data can have Null values, False otherwise
            table_transformer_style: 'gan' or 'cube'

    Returns:
        table_tranformer (TableTransformer) to pre and post-process the data
    """
    col_categories = _get_column_by_types(metadata, query_json.select_cols)
    style = query_json.table_transformer_style
    nullable = query_json.nullable

    constraints = {}
    for col in col_categories[SSynthColumnType.PRIVATE_ID]:
        constraints[col] = AnonymizationTransformer(SSYNTH_PRIVATE_COLUMN)

    if style == SSynthTableTransStyle.GAN:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = MinMaxTransformer(
                lower=metadata["columns"][col]["lower"],
                upper=metadata["columns"][col]["upper"],
                nullable=nullable,
            )
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    MinMaxTransformer(
                        lower=metadata["columns"][col]["lower"],
                        upper=metadata["columns"][col]["upper"],
                        nullable=nullable
                    ),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = ChainTransformer(
                [LabelTransformer(nullable=nullable), OneHotEncoder()]
            )
    else:
        for col in col_categories[SSynthColumnType.CATEGORICAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.CONTINUOUS]:
            constraints[col] = BinTransformer(nullable=nullable)
        for col in col_categories[SSynthColumnType.DATETIME]:
            constraints[col] = ChainTransformer(
                [
                    DateTimeTransformer(),
                    BinTransformer(bins=20, nullable=nullable),
                ]
            )
        for col in col_categories[SSynthColumnType.ORDINAL]:
            constraints[col] = LabelTransformer(nullable=nullable)
    print(constraints)
    return TableTransformer.create(
        data=private_data,
        style=style,
        nullable=nullable,
        constraints=constraints,
    )

## Synthesize data

In [46]:
from snsynth.transform import ChainTransformer, LabelTransformer, MinMaxTransformer, OneHotEncoder
from smartnoise_synth_logger import serialise_constraints

constraints = {
   'species': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'island': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
   'bill_length_mm': MinMaxTransformer(lower=30.0, upper=65.0, nullable=True),
   'bill_depth_mm': MinMaxTransformer(lower=13.0, upper=23.0, nullable=True),
   'flipper_length_mm': MinMaxTransformer(lower=150.0, upper=250.0, nullable=True),
   'body_mass_g': MinMaxTransformer(lower=2000.0, upper=7000.0, nullable=True),
   'sex': ChainTransformer([LabelTransformer(nullable=True), OneHotEncoder()]),
}

In [47]:
serialise_constraints(constraints)

'{"module": "smartnoise-synth", "version": "1.0.4", "constraints": {"species": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "island": {"type": "_ssynth_transformer:ChainTransformer", "params": [{"type": "_ssynth_transformer:LabelTransformer", "params": {"nullable": true}}, {"type": "_ssynth_transformer:OneHotEncoder", "params": {}}]}, "bill_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 30.0, "upper": 65.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "bill_depth_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 13.0, "upper": 23.0, "epsilon": 0.0, "negative": true, "nullable": true, "odometer": null}}, "flipper_length_mm": {"type": "_ssynth_transformer:MinMaxTransformer", "params": {"lower": 150.0, "upper": 250.0, "epsilon": 0.0, "negative": tr

In [ ]:
query_json_penguin = {
    "dataset_name": "PENGUIN",
    "synth_name": "dpctgan",
    "epsilon": 0.1,
    "delta": 0.00001,
    "select_cols": ["bill_length_mm"],
    "synth_params": {
        "embedding_dim": 128, 
        "generator_dim": (256, 256), 
        "discriminator_dim": (256, 256),
        "batch_size": 50
    },
    "mul_matrix": [],
    "nullable": True,
    "table_transformer_style": "gan",
}
query_json_penguin = SmartnoiseSynthModel.model_validate(query_json_penguin)

In [ ]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    verbose=True,
    **query_json_penguin.synth_params,
)
model

In [ ]:
penguin_private_data = _preprocess_data(penguin_private_data, query_json_penguin)

In [ ]:
penguin_transformer = _prepare_data_transformer(penguin_metadata, penguin_private_data, query_json_penguin)
penguin_transformer.transformers

In [ ]:
penguin_encoded = penguin_transformer.fit_transform(penguin_private_data, epsilon=0.0)
penguin_transformer.odometer.spent

In [ ]:
model = Synthesizer.create(
    synth=query_json_penguin.synth_name,
    epsilon=query_json_penguin.epsilon,
    **query_json_penguin.synth_params,
)

In [ ]:
model.fit(
    data=penguin_private_data,
    transformer=penguin_transformer,
    preprocessor_eps=0.0, #0.0,  # will error if not 0.
    nullable=query_json_penguin.nullable,
)

In [ ]:
samples = model.sample(5)
print(samples)

## Serialise

In [45]:
import pickle
from base64 import b64encode
def serialise_model(model) -> str:
    serialised = b64encode(pickle.dumps(model))
    return serialised.decode("utf-8")

In [57]:
str_model = serialise_model(model)
str_model

In [59]:
b64_model = base64.b64decode(str_model)
b64_model

In [ ]:
new_model = pickle.loads(b64_model)
new_model

In [60]:
samples = new_model.sample(5)
print(samples)

   bill_length_mm
0       46.956818
1       45.232080
2       55.124942
3       50.191850
4       57.324367


In [63]:
import json
j_obj = json.dumps(model)

TypeError: Object of type DPCTGAN is not JSON serializable

In [46]:
import base64
import json
from base64 import b64encode
pickled_model = b64encode(pickle.dumps(model))
depickled_model = base64.b64decode(pickled_model)
json_model = json.loads(depickled_model)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
samples = json_model.sample(100)
print(samples)